In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Load asthma dataset and perform exploratory data analysis

In [2]:
asthma = pd.read_csv('C:\\Users\\Mary\\CIS5898\\FIT_capstone\\Original_datasets\\U.S._Chronic_Disease_Indicators__Asthma (1).csv') 
print(asthma.info()) #provides basic information on data
print(asthma.ndim) #provides dimensions
print(asthma.shape) #provides number of rows and number of columns 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64719 entries, 0 to 64718
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   YearStart                  64719 non-null  int64  
 1   YearEnd                    64719 non-null  int64  
 2   LocationAbbr               64719 non-null  object 
 3   LocationDesc               64719 non-null  object 
 4   DataSource                 64719 non-null  object 
 5   Topic                      64719 non-null  object 
 6   Question                   64719 non-null  object 
 7   Response                   0 non-null      float64
 8   DataValueUnit              57171 non-null  object 
 9   DataValueType              64719 non-null  object 
 10  DataValue                  32185 non-null  float64
 11  DataValueAlt               32185 non-null  float64
 12  DataValueFootnoteSymbol    33660 non-null  object 
 13  DatavalueFootnote          33660 non-null  obj

In [3]:
asthma['YearStart'].unique()

array([2012, 2014, 2011, 2015, 2013, 2017, 2010, 2016, 2019, 2018],
      dtype=int64)

In [4]:
#2012 is the only year in the dataset that matches a year in the air quality studies.
asthma2012 = asthma[asthma['YearStart']==2012].sort_values('LocationAbbr')
print(len(asthma2012))
asthma2012.head(5)

5902


,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
49980,2012,2012,AK,Alaska,BRFSS,Asthma,Pneumococcal vaccination among noninstitutiona...,NaN,%,Age-adjusted Prevalence,...,2,AST,AST6_1,AGEADJPREV,RACE,MRC,NaN,NaN,NaN,NaN
2901,2012,2012,AK,Alaska,NVSS,Asthma,Asthma mortality rate,NaN,"cases per 1,000,000",Crude Rate,...,2,AST,AST4_1,CRDRATE,OVERALL,OVR,NaN,NaN,NaN,NaN
46531,2012,2012,AK,Alaska,BRFSS,Asthma,Influenza vaccination among noninstitutionaliz...,NaN,%,Crude Prevalence,...,2,AST,AST5_2,CRDPREV,RACE,HIS,NaN,NaN,NaN,NaN
37486,2012,2012,AK,Alaska,BRFSS,Asthma,Asthma prevalence among women aged 18-44 years,NaN,%,Crude Prevalence,...,2,AST,AST1_2,CRDPREV,RACE,OTH,NaN,NaN,NaN,NaN
36772,2012,2012,AK,Alaska,BRFSS,Asthma,Influenza vaccination among noninstitutionaliz...,NaN,%,Crude Prevalence,...,2,AST,AST5_1,CRDPREV,GENDER,GENM,NaN,NaN,NaN,NaN


In [5]:
#find which states are represented
asthma2012['LocationDesc'].unique()


array(['Alaska', 'Alabama', 'Arkansas', 'Arizona', 'California',
       'Colorado', 'Connecticut', 'District of Columbia', 'Delaware',
       'Florida', 'Georgia', 'Guam', 'Hawaii', 'Iowa', 'Idaho',
       'Illinois', 'Indiana', 'Kansas', 'Kentucky', 'Louisiana',
       'Massachusetts', 'Maryland', 'Maine', 'Michigan', 'Minnesota',
       'Missouri', 'Mississippi', 'Montana', 'North Carolina',
       'North Dakota', 'Nebraska', 'New Hampshire', 'New Jersey',
       'New Mexico', 'Nevada', 'New York', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'United States', 'Utah',
       'Virginia', 'Virgin Islands', 'Vermont', 'Washington', 'Wisconsin',
       'West Virginia', 'Wyoming'], dtype=object)

### Clean dataset

In [15]:
#drop District of Columbia, Guam, Puerto Rico, Virgin Islands, and United States
asthma2012States = asthma2012[(asthma2012['LocationDesc']!= 'District of Columbia') 
                & (asthma2012['LocationDesc']!='Guam')
                & (asthma2012['LocationDesc']!='Puerto Rico') 
                & (asthma2012['LocationDesc']!='United States')               
                & (asthma2012['LocationDesc']!='Virgin Islands') ]
asthma2012States

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
49980,2012,2012,AK,Alaska,BRFSS,Asthma,Pneumococcal vaccination among noninstitutiona...,NaN,%,Age-adjusted Prevalence,...,2,AST,AST6_1,AGEADJPREV,RACE,MRC,NaN,NaN,NaN,NaN
2901,2012,2012,AK,Alaska,NVSS,Asthma,Asthma mortality rate,NaN,"cases per 1,000,000",Crude Rate,...,2,AST,AST4_1,CRDRATE,OVERALL,OVR,NaN,NaN,NaN,NaN
46531,2012,2012,AK,Alaska,BRFSS,Asthma,Influenza vaccination among noninstitutionaliz...,NaN,%,Crude Prevalence,...,2,AST,AST5_2,CRDPREV,RACE,HIS,NaN,NaN,NaN,NaN
37486,2012,2012,AK,Alaska,BRFSS,Asthma,Asthma prevalence among women aged 18-44 years,NaN,%,Crude Prevalence,...,2,AST,AST1_2,CRDPREV,RACE,OTH,NaN,NaN,NaN,NaN
36772,2012,2012,AK,Alaska,BRFSS,Asthma,Influenza vaccination among noninstitutionaliz...,NaN,%,Crude Prevalence,...,2,AST,AST5_1,CRDPREV,GENDER,GENM,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37910,2012,2012,WY,Wyoming,BRFSS,Asthma,Influenza vaccination among noninstitutionaliz...,NaN,%,Age-adjusted Prevalence,...,56,AST,AST5_2,AGEADJPREV,RACE,MRC,NaN,NaN,NaN,NaN
38017,2012,2012,WY,Wyoming,BRFSS,Asthma,Influenza vaccination among noninstitutionaliz...,NaN,%,Crude Prevalence,...,56,AST,AST5_1,CRDPREV,RACE,MRC,NaN,NaN,NaN,NaN
64694,2012,2012,WY,Wyoming,BRFSS,Asthma,Pneumococcal vaccination among noninstitutiona...,NaN,%,Crude Prevalence,...,56,AST,AST6_2,CRDPREV,RACE,BLK,NaN,NaN,NaN,NaN
35877,2012,2012,WY,Wyoming,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Age-adjusted Prevalence,...,56,AST,AST1_1,AGEADJPREV,RACE,BLK,NaN,NaN,NaN,NaN


In [16]:
#find questions
asthma2012States['Question'].unique()

array(['Pneumococcal vaccination among noninstitutionalized adults aged 18-64 years with asthma',
       'Asthma mortality rate',
       'Influenza vaccination among noninstitutionalized adults aged >= 65 years with asthma',
       'Asthma prevalence among women aged 18-44 years',
       'Influenza vaccination among noninstitutionalized adults aged 18-64 years with asthma',
       'Pneumococcal vaccination among noninstitutionalized adults aged >= 65 years with asthma',
       'Current asthma prevalence among adults aged >= 18 years'],
      dtype=object)

In [17]:
#the question of interest is the current asthma prevalence among adults aged >=18 years
asthma2012CurrentAsthmaPrevalence = asthma2012States[(asthma2012States['Question']=='Current asthma prevalence among adults aged >= 18 years')]
print(len(asthma2012CurrentAsthmaPrevalence))
asthma2012CurrentAsthmaPrevalence.head()

800


,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
23452,2012,2012,AK,Alaska,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,2,AST,AST1_1,CRDPREV,GENDER,GENM,NaN,NaN,NaN,NaN
23207,2012,2012,AK,Alaska,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Age-adjusted Prevalence,...,2,AST,AST1_1,AGEADJPREV,RACE,OTH,NaN,NaN,NaN,NaN
23193,2012,2012,AK,Alaska,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,2,AST,AST1_1,CRDPREV,GENDER,GENF,NaN,NaN,NaN,NaN
23161,2012,2012,AK,Alaska,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,2,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
23149,2012,2012,AK,Alaska,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Age-adjusted Prevalence,...,2,AST,AST1_1,AGEADJPREV,RACE,BLK,NaN,NaN,NaN,NaN


In [18]:
#overall incidence (not stratified by gender or race) is the column of interest for stratification category
#limit the dataset to overall incidence
asthma2012CAPOverall = asthma2012CurrentAsthmaPrevalence[asthma2012CurrentAsthmaPrevalence['StratificationCategoryID1'] == 'OVERALL']
asthma2012CAPOverall

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
23161,2012,2012,AK,Alaska,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,2,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
22895,2012,2012,AK,Alaska,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Age-adjusted Prevalence,...,2,AST,AST1_1,AGEADJPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
23239,2012,2012,AL,Alabama,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Age-adjusted Prevalence,...,1,AST,AST1_1,AGEADJPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
23038,2012,2012,AL,Alabama,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,1,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
22988,2012,2012,AR,Arkansas,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,5,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28568,2012,2012,WI,Wisconsin,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,55,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
36056,2012,2012,WV,West Virginia,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Age-adjusted Prevalence,...,54,AST,AST1_1,AGEADJPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
33408,2012,2012,WV,West Virginia,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,54,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
28448,2012,2012,WY,Wyoming,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,56,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN


In [19]:
asthma2012CAPOverall

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
23161,2012,2012,AK,Alaska,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,2,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
22895,2012,2012,AK,Alaska,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Age-adjusted Prevalence,...,2,AST,AST1_1,AGEADJPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
23239,2012,2012,AL,Alabama,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Age-adjusted Prevalence,...,1,AST,AST1_1,AGEADJPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
23038,2012,2012,AL,Alabama,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,1,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
22988,2012,2012,AR,Arkansas,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,5,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28568,2012,2012,WI,Wisconsin,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,55,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
36056,2012,2012,WV,West Virginia,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Age-adjusted Prevalence,...,54,AST,AST1_1,AGEADJPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
33408,2012,2012,WV,West Virginia,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,54,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
28448,2012,2012,WY,Wyoming,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,56,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN


In [20]:
asthma2012CAPOverall.iloc[0:10, 0:18]

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,DataValue,DataValueAlt,DataValueFootnoteSymbol,DatavalueFootnote,LowConfidenceLimit,HighConfidenceLimit,StratificationCategory1,Stratification1
23161,2012,2012,AK,Alaska,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,9.0,9.0,NaN,NaN,7.9,10.3,Overall,Overall
22895,2012,2012,AK,Alaska,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Age-adjusted Prevalence,9.1,9.1,NaN,NaN,7.9,10.3,Overall,Overall
23239,2012,2012,AL,Alabama,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Age-adjusted Prevalence,8.5,8.5,NaN,NaN,7.7,9.5,Overall,Overall
23038,2012,2012,AL,Alabama,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,8.6,8.6,NaN,NaN,7.8,9.5,Overall,Overall
22988,2012,2012,AR,Arkansas,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,8.8,8.8,NaN,NaN,7.8,9.8,Overall,Overall
23253,2012,2012,AR,Arkansas,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Age-adjusted Prevalence,8.7,8.7,NaN,NaN,7.6,9.8,Overall,Overall
23310,2012,2012,AZ,Arizona,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Age-adjusted Prevalence,8.7,8.7,NaN,NaN,7.7,9.8,Overall,Overall
23560,2012,2012,AZ,Arizona,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,8.6,8.6,NaN,NaN,7.7,9.7,Overall,Overall
23420,2012,2012,CA,California,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Age-adjusted Prevalence,8.8,8.8,NaN,NaN,8.2,9.4,Overall,Overall
23261,2012,2012,CA,California,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,8.8,8.8,NaN,NaN,8.2,9.5,Overall,Overall


In [21]:
#limit dataset to crude prevalence
asthma2012CAPOverallCrude = asthma2012CAPOverall[asthma2012CAPOverall['DataValueType']=='Crude Prevalence']
asthma2012CAPOverallCrude

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
23161,2012,2012,AK,Alaska,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,2,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
23038,2012,2012,AL,Alabama,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,1,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
22988,2012,2012,AR,Arkansas,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,5,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
23560,2012,2012,AZ,Arizona,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,4,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
23261,2012,2012,CA,California,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,6,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
23481,2012,2012,CO,Colorado,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,8,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
32659,2012,2012,CT,Connecticut,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,9,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
26619,2012,2012,DE,Delaware,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,10,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
22673,2012,2012,FL,Florida,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,12,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN
30860,2012,2012,GA,Georgia,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,...,13,AST,AST1_1,CRDPREV,OVERALL,OVR,NaN,NaN,NaN,NaN


In [22]:
len(asthma2012CAPOverallCrude)

50

### What does the reported incidence of asthma look like by state?

In [24]:
asthma2012Subset = asthma2012CAPOverallCrude.iloc[:,0:18]
asthma2012Subset.sort_values('DataValue', ascending = False)

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,DataValue,DataValueAlt,DataValueFootnoteSymbol,DatavalueFootnote,LowConfidenceLimit,HighConfidenceLimit,StratificationCategory1,Stratification1
26078,2012,2012,ME,Maine,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,11.1,11.1,NaN,NaN,10.3,11.9,Overall,Overall
28646,2012,2012,KY,Kentucky,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,11.1,11.1,NaN,NaN,10.2,12.0,Overall,Overall
32795,2012,2012,VT,Vermont,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.9,10.9,NaN,NaN,9.9,12.1,Overall,Overall
23605,2012,2012,MA,Massachusetts,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.8,10.8,NaN,NaN,10.2,11.5,Overall,Overall
36314,2012,2012,RI,Rhode Island,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.8,10.8,NaN,NaN,9.7,12.0,Overall,Overall
25260,2012,2012,OR,Oregon,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.6,10.6,NaN,NaN,9.5,11.8,Overall,Overall
27412,2012,2012,MI,Michigan,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.5,10.5,NaN,NaN,9.7,11.4,Overall,Overall
25887,2012,2012,OH,Ohio,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.5,10.5,NaN,NaN,9.7,11.2,Overall,Overall
35720,2012,2012,MO,Missouri,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.4,10.4,NaN,NaN,9.4,11.5,Overall,Overall
33336,2012,2012,NH,New Hampshire,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.2,10.2,NaN,NaN,9.2,11.3,Overall,Overall


### Which states have the highest and lowest reported incidence of asthma?

In [25]:
#highest
asthma2012Highest = asthma2012Subset.sort_values('DataValue', ascending = False).head(10)

asthma2012Highest

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,DataValue,DataValueAlt,DataValueFootnoteSymbol,DatavalueFootnote,LowConfidenceLimit,HighConfidenceLimit,StratificationCategory1,Stratification1
26078,2012,2012,ME,Maine,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,11.1,11.1,NaN,NaN,10.3,11.9,Overall,Overall
28646,2012,2012,KY,Kentucky,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,11.1,11.1,NaN,NaN,10.2,12.0,Overall,Overall
32795,2012,2012,VT,Vermont,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.9,10.9,NaN,NaN,9.9,12.1,Overall,Overall
23605,2012,2012,MA,Massachusetts,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.8,10.8,NaN,NaN,10.2,11.5,Overall,Overall
36314,2012,2012,RI,Rhode Island,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.8,10.8,NaN,NaN,9.7,12.0,Overall,Overall
25260,2012,2012,OR,Oregon,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.6,10.6,NaN,NaN,9.5,11.8,Overall,Overall
27412,2012,2012,MI,Michigan,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.5,10.5,NaN,NaN,9.7,11.4,Overall,Overall
25887,2012,2012,OH,Ohio,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.5,10.5,NaN,NaN,9.7,11.2,Overall,Overall
35720,2012,2012,MO,Missouri,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.4,10.4,NaN,NaN,9.4,11.5,Overall,Overall
33336,2012,2012,NH,New Hampshire,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,10.2,10.2,NaN,NaN,9.2,11.3,Overall,Overall


In [26]:
#lowest
asthma2012Lowest = asthma2012Subset.sort_values('DataValue').head(10)
asthma2012Lowest

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,DataValue,DataValueAlt,DataValueFootnoteSymbol,DatavalueFootnote,LowConfidenceLimit,HighConfidenceLimit,StratificationCategory1,Stratification1
31164,2012,2012,TX,Texas,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,6.8,6.8,NaN,NaN,6.1,7.6,Overall,Overall
31996,2012,2012,NE,Nebraska,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,7.4,7.4,NaN,NaN,6.9,7.9,Overall,Overall
34758,2012,2012,NV,Nevada,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,7.4,7.4,NaN,NaN,6.4,8.5,Overall,Overall
32098,2012,2012,LA,Louisiana,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,7.4,7.4,NaN,NaN,6.6,8.3,Overall,Overall
30559,2012,2012,SD,South Dakota,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,7.5,7.5,NaN,NaN,6.6,8.5,Overall,Overall
33351,2012,2012,ND,North Dakota,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,7.6,7.6,NaN,NaN,6.6,8.7,Overall,Overall
25317,2012,2012,NC,North Carolina,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,7.7,7.7,NaN,NaN,7.0,8.4,Overall,Overall
35818,2012,2012,TN,Tennessee,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,7.7,7.7,NaN,NaN,6.8,8.7,Overall,Overall
26385,2012,2012,MN,Minnesota,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,8.0,8.0,NaN,NaN,7.3,8.6,Overall,Overall
37009,2012,2012,IA,Iowa,BRFSS,Asthma,Current asthma prevalence among adults aged >=...,NaN,%,Crude Prevalence,8.1,8.1,NaN,NaN,7.3,9.0,Overall,Overall


In [27]:
#What are summary statistics on the cleaned subset?
asthma2012Subset.describe()

,YearStart,YearEnd,Response,DataValue,DataValueAlt,LowConfidenceLimit,HighConfidenceLimit
count,50.0,50.0,0.0,50.000000,50.000000,50.000000,50.000000
mean,2012.0,2012.0,NaN,9.054000,9.054000,8.192000,9.988000
std,0.0,0.0,NaN,1.120716,1.120716,1.107275,1.162973
min,2012.0,2012.0,NaN,6.800000,6.800000,6.100000,7.600000
25%,2012.0,2012.0,NaN,8.250000,8.250000,7.300000,9.200000
50%,2012.0,2012.0,NaN,8.900000,8.900000,7.950000,9.800000
75%,2012.0,2012.0,NaN,10.050000,10.050000,9.175000,10.975000
max,2012.0,2012.0,NaN,11.100000,11.100000,10.300000,12.100000
